# Import Data to PostgreSQL: `drug_db.sql`

In this second section, we'll use the cleaned data of `DrugReviews_cleaned.csv` to be imported to PostgreSQL.  They're open source, rich of features, and also it's able to migrate easily with other databases (dB).

The process are done on pgAdmin 4 on PostgreSQL 14. However, for the purpose of the notebook, we'll be utilizing module of `psycopg2` to integrate Python to PostgreSQL. 

Mainly people use `sqlalchemy` to showcase the data of query, but here we'll use `pandas` for easy data interface in form of DataFrame.

## `CREATE` `DATABASE` and `SCHEMA`

In [1]:
# !pip install psycopg2

In [2]:
import psycopg2
import pandas as pd

user_login = "host=localhost port=5432 user='postgres' password='postgres'"

In [3]:
conn = psycopg2.connect(user_login)

# Set autocommit to True on non-data query commands
conn.autocommit = True

cur = conn.cursor()

query_database = 'CREATE DATABASE drug_db;'
cur.execute(query_database)
conn.close()

In [4]:
db_login = "host=localhost port=5432 dbname='drug_db' user='postgres' password='postgres'"

In [5]:
conn = psycopg2.connect(db_login)

# Set autocommit to True on non-query commands
conn.autocommit = True

cur = conn.cursor()

query_schema   = 'CREATE SCHEMA drug_rev;'
cur.execute(query_schema)

conn.close()

## `CREATE TABLE` : `drug_reviews`

In [6]:
query_table = '''
    CREATE TABLE IF NOT EXISTS drug_rev.drug_reviews (
        MedicineUsedFor VARCHAR(100),
        MedicineBrandName VARCHAR(100),
        MedicineGenericName VARCHAR(100),
        ReviewDate DATE,
        UserName VARCHAR(20),
        IntakeTime VARCHAR(50),
        Reviews text,
        ReviewLength INTEGER,
        Rating INTEGER,
        NumberOfLikes INTEGER
    );'''

In [7]:
conn = psycopg2.connect(db_login)
cur = conn.cursor()

# Set autocommit to False for other commands
conn.autocommit = False

cur.execute(query_table)
conn.commit()
conn.close()

Note: When executing commands in `psycopg2`, the executions are usually done in a translating block, meaning it will only run after calling the method `connection.commit()` and will be applied if and only if all of the queries in it are successful.

This will prevent overwriting data when executing commands such as `UPDATE` or `DELETE`, ones that can be vulnerable to the database.

## `COPY` ... `FROM DrugReviews_cleaned.csv`

In this section you may see an error when the data is in `Local Disk C:`. You could either:
* Move the data into other local disk storage, or
* Allow `Full Control` Permissions for Everyone (Tutorial: [StackOverflow](https://stackoverflow.com/a/65459173))

In [8]:
## Double all slashes '\' to prevent UnicodeError 
## caused by character '\U' in C:\Users

query_copy = """
    -- Copy all data from 'DrugReviews_cleaned.csv' to PostgreSQL
    COPY drug_rev.drug_reviews(
        MedicineUsedFor, MedicineBrandName, MedicineGenericName,
        ReviewDate, UserName, IntakeTime, Reviews, ReviewLength,
        Rating, NumberOfLikes)
    FROM 'C:\\Users\\acer\\Downloads\\PROJECTS-2.0\\DrugReviews\\DrugReviews_cleaned.csv'
    DELIMITER ','
    CSV HEADER;
"""

In [9]:
query_check = '''
    SELECT *
      FROM drug_rev.drug_reviews
     LIMIT 5;
'''

In [10]:
conn = psycopg2.connect(db_login)
cur = conn.cursor()

cur.execute(query_copy)
conn.commit()

P.S. [`pandas` support `psycopg2`](https://docs.sqlalchemy.org/en/20/core/engines.html#postgresql) as one of connections of PostgreSQL in `sqlalchemy`. So no need to worry about the warnings.

In [11]:
conn = psycopg2.connect(db_login)
cur = conn.cursor()

cur.execute(query_check)
conn.commit()

df01 = pd.read_sql_query(query_check, conn)
df01

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\3525517839.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df01 = pd.read_sql_query(query_check, conn)


,medicineusedfor,medicinebrandname,medicinegenericname,reviewdate,username,intaketime,reviews,reviewlength,rating,numberoflikes
0,Cough,Acetaminophen / Codeine,Not Mentioned,2008-04-01,smoore,Not Specified,Works good as a cough suppressant.,34,9,24
1,Cough,Benzonatate,Not Mentioned,2008-04-01,Anonymous,Not Specified,Pneumonia cough was non-stop - gave almost ins...,210,9,39
2,Dermatologic Lesion,Methylprednisolone Dose Pack,Methylprednisolone,2008-04-01,Anonymous,Not Specified,This steriod helped kill the pain of my condit...,162,8,24
3,"Hypogonadism, Male",Androgel,Not Mentioned,2008-04-01,MikeC...,Not Specified,I'm a 35 year old male and I had no idea that ...,105,9,380
4,Depression,Celexa,Not Mentioned,2008-04-01,Cherpie,Not Specified,It is so nice to have my life back!!!,37,10,206


## `ALTER TABLE` `drug_reviews`

To implement calculation in ratings such as average and standard deviations, we need to change the datatype of column `Rating` from the existing `INTEGER` to `FLOAT`

In [12]:
query_alter_datatype = '''
    -- Change datatype of rating to enable calculations of AVG and STD
    ALTER TABLE drug_rev.drug_reviews
        ALTER COLUMN Rating
        TYPE FLOAT;
'''

In [13]:
cur.execute(query_alter_datatype)
conn.commit()

query_check_metadata = '''
    SELECT *
      FROM INFORMATION_SCHEMA.COLUMNS
     WHERE table_name = 'drug_reviews';
'''

df02 = pd.read_sql_query(query_check_metadata, conn)
df02

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\2559259649.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df02 = pd.read_sql_query(query_check_metadata, conn)


,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,drug_db,drug_rev,drug_reviews,reviewdate,4,None,YES,date,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,drug_db,drug_rev,drug_reviews,reviewlength,8,None,YES,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,drug_db,drug_rev,drug_reviews,rating,9,None,YES,double precision,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,drug_db,drug_rev,drug_reviews,numberoflikes,10,None,YES,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,drug_db,drug_rev,drug_reviews,medicineusedfor,1,None,YES,character varying,100.0,4.000000e+02,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,drug_db,drug_rev,drug_reviews,reviews,7,None,YES,text,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,drug_db,drug_rev,drug_reviews,intaketime,6,None,YES,character varying,50.0,2.000000e+02,...,NO,None,None,None,None,None,NO,NEVER,None,YES
7,drug_db,drug_rev,drug_reviews,medicinebrandname,2,None,YES,character varying,100.0,4.000000e+02,...,NO,None,None,None,None,None,NO,NEVER,None,YES
8,drug_db,drug_rev,drug_reviews,medicinegenericname,3,None,YES,character varying,100.0,4.000000e+02,...,NO,None,None,None,None,None,NO,NEVER,None,YES
9,drug_db,drug_rev,drug_reviews,username,5,None,YES,character varying,20.0,8.000000e+01,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [14]:
conn.close()

As we can see on `column_name`, `ordinal_position`, and `data_type`, the metadata on columns matches the `query_table` written above.

It also matches with the alteration of `Rating`, with `data_type` as `double precision`.

## `SELECT ...`: Explore Data with Queries

In [15]:
# Login information from user to schema:
complete_login = """
    host=localhost port=5432 dbname='drug_db' 
    user='postgres' password='postgres' options='-c search_path=drug_rev'
"""

In [16]:
conn = psycopg2.connect(complete_login)
cur = conn.cursor()

### No of Reviews: Daily, Monthly, Annually

In [17]:
query_no_of_reviews_daily = """
    SELECT ReviewDate, COUNT(ReviewDate) AS count_per_date
      FROM drug_reviews
     GROUP BY ReviewDate
     ORDER BY ReviewDate;
"""

In [18]:
query_no_of_reviews_monthly = """
    SELECT to_char(date_trunc('month', ReviewDate), 'YYYY-MM') AS month,
           COUNT(ReviewDate) AS count_per_month
      FROM drug_reviews
     GROUP BY month
     ORDER BY month;
"""

In [19]:
query_no_of_reviews_annually = """
    SELECT to_char(date_trunc('year', ReviewDate), 'YYYY') AS year,
           COUNT(ReviewDate) AS count_per_year
      FROM drug_reviews
     GROUP BY year
     ORDER BY year;
"""

In [20]:
cur.execute(query_no_of_reviews_daily)
conn.commit()

df03 = pd.read_sql_query(query_no_of_reviews_daily, conn)
df03

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\1996006293.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df03 = pd.read_sql_query(query_no_of_reviews_daily, conn)


,reviewdate,count_per_date
0,2008-02-24,4
1,2008-02-25,22
2,2008-02-26,23
3,2008-02-27,20
4,2008-02-28,7
...,...,...
5627,2023-08-03,28
5628,2023-08-04,30
5629,2023-08-05,18
5630,2023-08-06,4


In [21]:
cur.execute(query_no_of_reviews_monthly)
conn.commit()

df04 = pd.read_sql_query(query_no_of_reviews_monthly, conn)
df04

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\2026197954.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df04 = pd.read_sql_query(query_no_of_reviews_monthly, conn)


,month,count_per_month
0,2008-02,83
1,2008-03,462
2,2008-04,414
3,2008-05,353
4,2008-06,374
...,...,...
182,2023-04,1704
183,2023-05,1714
184,2023-06,1499
185,2023-07,945


In [22]:
cur.execute(query_no_of_reviews_annually)
conn.commit()

df05 = pd.read_sql_query(query_no_of_reviews_annually, conn)
df05

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\61872982.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df05 = pd.read_sql_query(query_no_of_reviews_annually, conn)


,year,count_per_year
0,2008,3965
1,2009,8991
2,2010,6262
3,2011,8681
4,2012,7622
5,2013,9568
6,2014,9589
7,2015,21909
8,2016,28996
9,2017,24308


### Top 20 Review and Rating Statistics per Brand

In [23]:
query_order_by_no_reviews = '''
    -- Check the highest brand of each usage in rating
    SELECT MedicineBrandName, MedicineUsedFor,
           COUNT(Reviews) AS count_reviews, 
           AVG(Rating) AS avg_rating, 
           STDDEV(Rating) AS std_rating
      FROM drug_reviews
     GROUP BY MedicineBrandName, MedicineUsedFor
     ORDER BY count_reviews DESC, avg_rating DESC, std_rating
     LIMIT 20;
'''

In [24]:
cur.execute(query_order_by_no_reviews)
conn.commit()

df06 = pd.read_sql_query(query_order_by_no_reviews, conn)
df06

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\3940633171.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df06 = pd.read_sql_query(query_order_by_no_reviews, conn)


,medicinebrandname,medicineusedfor,count_reviews,avg_rating,std_rating
0,Nexplanon,Birth Control,4536,5.075397,3.348162
1,Plan B One-Step,Emergency Contraception,2489,8.585376,2.680322
2,Phentermine,Weight Loss (Obesity/Overweight),1859,8.649274,2.091210
3,Mirena,Birth Control,1813,6.468836,3.380365
4,Miconazole Topical,Vaginal Yeast Infection,1758,2.541524,2.675004
5,Contrave,Weight Loss (Obesity/Overweight),1726,6.585747,3.374655
6,Implanon,Birth Control,1570,6.061783,3.315472
7,Suprep Bowel Prep Kit,Bowel Preparation,1473,7.164291,3.113592
8,Kyleena,Birth Control,1441,6.512838,3.159643
9,Lo Loestrin Fe,Birth Control,1437,5.647878,3.468504


### Top 20 Review and Rating Statistics per Drug Usage

In [25]:
query_avg_rating_per_usage = '''
    -- Check the average rating and total reviews by medicine usage
    SELECT MedicineUsedFor, 
           COUNT(Reviews) AS count_reviews, 
           AVG(Rating) AS avg_rating, 
           STDDEV(Rating) AS std_rating
      FROM drug_reviews
     GROUP BY MedicineUsedFor
     ORDER BY count_reviews DESC, avg_rating DESC, std_rating
     LIMIT 20;
'''

In [26]:
cur.execute(query_avg_rating_per_usage)
conn.commit()

df07 = pd.read_sql_query(query_avg_rating_per_usage, conn)
df07

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\1986198518.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df07 = pd.read_sql_query(query_avg_rating_per_usage, conn)


,medicineusedfor,count_reviews,avg_rating,std_rating
0,Birth Control,35386,5.441672,3.444163
1,Depression,12154,6.750617,3.349166
2,Anxiety,9701,7.110813,3.361637
3,Weight Loss (Obesity/Overweight),8846,7.685281,2.990882
4,Pain,7276,6.852117,3.453429
5,Acne,6924,6.916811,3.283338
6,Insomnia,6737,6.122458,3.660349
7,Vaginal Yeast Infection,5985,3.649624,3.409993
8,Bipolar Disorder,5699,6.804527,3.373608
9,"Diabetes, Type 2",4818,5.817767,3.560503


### Top 5 Brands and Generics on Top 20 Usages

In this query, we're picking top 5 since there are a good amount of drugs that are used by many, but has less average ratings. 

The author thinks top 5 is a good data for consideration of trade-offs and at last decision by the pharma stakeholders.

* **Top Brands** could give insights at retail pharmacy stores to stock on the most-used and/or high-rating drugs amongst the top 5 drugs, and 
* **Top Generics** could give insights at pharma research companies, either to keep the quality or as a feedback to improve the drug.

**Note:** 

At data cleaning, most drugs that don't include generic name would be named as `Not Mentioned`. Hence, the data will rise as one of top 5 in each drug usage. Therefore, in Top Generics query, we'll query **top 6** to take the *top 5 after*.

In [27]:
query_top_5_brands_on_top_20_usages = """

    SELECT dn.MedicineBrandName, dn.MedicineUsedFor,
           dn.count_reviews, dn.avg_rating, dn.std_rating, 
           du.rank_usages, dn.rank_brand
      FROM (

      -- Query 1: Data grouped by brands and usages, with rank of brand
        SELECT MedicineBrandName,
               MedicineUsedFor,
               COUNT(Reviews) AS count_reviews,
               AVG(Rating) AS avg_rating,
               STDDEV(Rating) AS std_rating,
               ROW_NUMBER() OVER (
                   PARTITION BY MedicineUsedFor
                   ORDER BY COUNT(Reviews) DESC) AS rank_brand
          FROM drug_reviews
         GROUP BY MedicineBrandName, MedicineUsedFor) AS dn	  

      -- Query 2: Data grouped by usages only, with rank of usage
      FULL JOIN (
        SELECT MedicineUsedFor, 
               ROW_NUMBER() OVER (
                   ORDER BY COUNT(Reviews) DESC) AS rank_usages
          FROM drug_reviews
         GROUP BY MedicineUsedFor
         ) AS du
         
     -- Both query joined by column `MedicineFor`
     USING (MedicineUsedFor)
     
     -- Conditions: Top 20 Usages >> Top 5 Brand for each
     WHERE du.rank_usages <= 20 AND dn.rank_brand <= 5
     ORDER BY du.rank_usages, dn.rank_brand;
"""

In [28]:
query_top_5_generics_on_top_20_usages = """

    SELECT gn.MedicineGenericName, gn.MedicineUsedFor,
           gn.count_reviews, gn.avg_rating, gn.std_rating, 
           du.rank_usages, gn.rank_generic
      FROM (
      
      -- Query 1: Data grouped by generics and usages, with rank of generic
        SELECT MedicineGenericName,
               MedicineUsedFor,
               COUNT(Reviews) AS count_reviews,
               AVG(Rating) AS avg_rating,
               STDDEV(Rating) AS std_rating,
               ROW_NUMBER() OVER (
                   PARTITION BY MedicineUsedFor
                   ORDER BY COUNT(Reviews) DESC) AS rank_generic
          FROM drug_reviews
         GROUP BY MedicineGenericName, MedicineUsedFor) AS gn  

      -- Query 2: Data grouped by usages only, with rank of usage
     FULL JOIN (
        SELECT MedicineUsedFor, 
               ROW_NUMBER() OVER (
                   ORDER BY COUNT(Reviews) DESC) AS rank_usages
          FROM drug_reviews
         GROUP BY MedicineUsedFor
         ) AS du
         
     -- Both query joined by column `MedicineFor`
     USING (MedicineUsedFor)
     
     -- Conditions: Top 20 Usages >> Top 6 Generics for each
     WHERE du.rank_usages <= 20 AND gn.rank_generic <= 6
     ORDER BY du.rank_usages, gn.rank_generic;

"""

In [29]:
cur.execute(query_top_5_brands_on_top_20_usages)
conn.commit()

df08 = pd.read_sql_query(query_top_5_brands_on_top_20_usages, conn)
df08

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\2721324157.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df08 = pd.read_sql_query(query_top_5_brands_on_top_20_usages, conn)


,medicinebrandname,medicineusedfor,count_reviews,avg_rating,std_rating,rank_usages,rank_brand
0,Nexplanon,Birth Control,4536,5.075397,3.348162,1,1
1,Mirena,Birth Control,1813,6.468836,3.380365,1,2
2,Implanon,Birth Control,1570,6.061783,3.315472,1,3
3,Kyleena,Birth Control,1441,6.512838,3.159643,1,4
4,Lo Loestrin Fe,Birth Control,1437,5.647878,3.468504,1,5
...,...,...,...,...,...,...,...
95,Oxycodone,Chronic Pain,311,5.842444,3.661165,20,1
96,Belbuca,Chronic Pain,212,4.349057,3.517168,20,2
97,Acetaminophen / Oxycodone,Chronic Pain,193,4.036269,3.451106,20,3
98,Cymbalta,Chronic Pain,185,5.702703,3.569686,20,4


In [30]:
cur.execute(query_top_5_generics_on_top_20_usages)
conn.commit()

df09 = pd.read_sql_query(query_top_5_generics_on_top_20_usages, conn)
df09

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\3187907545.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df09 = pd.read_sql_query(query_top_5_generics_on_top_20_usages, conn)


,medicinegenericname,medicineusedfor,count_reviews,avg_rating,std_rating,rank_usages,rank_generic
0,Not Mentioned,Birth Control,9029,5.232584,3.508070,1,1
1,Etonogestrel,Birth Control,6027,5.329351,3.365678,1,2
2,Ethinyl Estradiol / Norethindrone,Birth Control,5230,5.249713,3.449715,1,3
3,Levonorgestrel,Birth Control,3833,6.492304,3.283413,1,4
4,Ethinyl Estradiol / Norgestimate,Birth Control,3662,5.356363,3.407735,1,5
...,...,...,...,...,...,...,...
112,Oxycodone,Chronic Pain,263,6.030418,3.661100,20,2
113,Buprenorphine,Chronic Pain,185,6.075676,3.543933,20,3
114,Acetaminophen / Oxycodone,Chronic Pain,180,6.927778,3.330963,20,4
115,Morphine,Chronic Pain,90,7.244444,2.892498,20,5


In [31]:
conn.close()

## Revoke Public Privileges

In this section we will make two groups, entered by different users, with two different privileges.
* `readonly` can only `SELECT` the tables, not edit them (`INSERT, UPDATE, DELETE`), while
* `readwrite` can do both (`SELECT, INSERT, UPDATE, DELETE`)

In [32]:
conn = psycopg2.connect(complete_login)
cur = conn.cursor()

# Turn `autocommit` to True for convenience
conn.autocommit = True

# Delete the groups and users if they exist
# to execute the code smoothly each time
cur.execute('DROP GROUP IF EXISTS readonly;')
cur.execute('DROP GROUP IF EXISTS readwrite;')
cur.execute('DROP USER IF EXISTS data_analyst;')
cur.execute('DROP USER IF EXISTS data_scientist;')

### Create the `readonly` Group

In [33]:
cur.execute('CREATE GROUP readonly NOLOGIN;')
cur.execute('GRANT CONNECT ON DATABASE drug_db TO readonly;')

cur.execute('GRANT USAGE ON SCHEMA drug_rev TO readonly;')
cur.execute('GRANT SELECT ON ALL TABLES IN SCHEMA drug_rev TO readonly')

### Create the `readwrite` Group

In [34]:
cur.execute('CREATE GROUP readwrite NOLOGIN;')
cur.execute('GRANT CONNECT ON DATABASE drug_db TO readwrite')

cur.execute('GRANT USAGE ON SCHEMA drug_rev TO readwrite;')
cur.execute('GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA drug_rev TO readwrite;')

### Create One User for Each Group: `data_analyst` and `data_scientist`

In [35]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'sapphire';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'emerald';")
cur.execute("GRANT readwrite TO data_scientist;")

In [36]:
conn.close()

### Test the Database Setup

To look for database of user roles and  group privileges, we'll check the table of `pg_roles` and `information_schema.table_privileges` respectively. We'll look at the following columns:

In `pg_roles`:

* `rolname` : *"What's the username / group name?"*
* `rolsuper` : *"Is the user/group a super-user?"*
* `rolcreaterole` : *"Can the user/group create new users?"*
* `rolcreatedb` : *"Can the user/group create new databases?"*
* `rolcanlogin` : *"Can the user/group log in?" (`True` to users, `False` to groups)*

In `information_schema.table_privileges`:
* `grantee` : *"Who's the user?"*
* `privilege_type`: *"What can they do?"*

In [37]:
conn = psycopg2.connect(complete_login)
cur = conn.cursor()

In [38]:
query_check_users_and_groups = '''
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin
      FROM pg_roles
     WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
'''

query_check_privileges = '''
    SELECT grantee, privilege_type
      FROM information_schema.table_privileges
     WHERE grantee IN ('readonly', 'readwrite');
'''

In [39]:
df10 = pd.read_sql_query(query_check_users_and_groups, conn)
df10

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\387711595.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df10 = pd.read_sql_query(query_check_users_and_groups, conn)


,rolname,rolsuper,rolcreaterole,rolcreatedb,rolcanlogin
0,readonly,False,False,False,False
1,readwrite,False,False,False,False
2,data_analyst,False,False,False,True
3,data_scientist,False,False,False,True


In [40]:
df11 = pd.read_sql_query(query_check_privileges, conn)
df11

C:\Users\acer\AppData\Local\Temp\ipykernel_1740\2592245914.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df11 = pd.read_sql_query(query_check_privileges, conn)


,grantee,privilege_type
0,readonly,SELECT
1,readwrite,INSERT
2,readwrite,SELECT
3,readwrite,UPDATE
4,readwrite,DELETE


In [41]:
conn.close()

Do not forget to save the file in pgAdmin4. We'll use it for dashboard in the next section.

# Outro

And there we have it! A PostgreSQL database has been made! To summarize, we've created:
* database of `drug_db`, schema of `drug_rev`, and table of `drug_reviews`,
* copy data from `DrugReviews_cleaned.csv` to table `drug_reviews`,
* alter components of the table in `drug_reviews`,
* execute several queries on the table `drug_reviews`, 
* make two groups with different privileges of `readonly` and `readwrite`,
* assign two different users in respective groups `data_analyst` and `data_scientist`, 
* and lastly check both roles and user privileges in respective tables.